In [1]:
import pandas as pd
import numpy as np
import re
import string

In [2]:
df = pd.read_excel("Complaints dataset.xlsx")

In [3]:
df.head()

,OFFER_NAME,CUSTOMER_TYPE,COMPLAINT_TYPE,PRODUCT,CASE_LEVEL_1,CASE_LEVEL_2,CASE_LEVEL_3,CUSTOMER_GROUP,CURRENT_STATUS,ESCALATION_FLAG,...,CLOSE_GROUP,AGE_BRACKET,ACTUAL_COMPLAINT,CALLBACK_MECHANISM,RESOLUTION,RESOLUTION_DESCRIPTION,CASE_DESC,OPEN_GR,Unnamed: 22,Unnamed: 23
0,FTTH Home,CBU,Technical,Internet,Technical complaint,TC_INTERNET_CONFIGURATION,SNW-FTTH - WAN configuration / Tools,FTTH Home,Resolved,No,...,NaN,1,User Not Filled,NaN,NaN,NaN,إعادة برمجة من النظامby Sultan,NaN,NaN,NaN
1,NaN,CBU,Technical,Mobile,Technical complaint,TC_INTERNET_CONNEXION,4G - Frequent Disconnection,NaN,Resolved,Yes,...,Mobile_BO,1,Actual Complaint,SMS,Coverage script was sent to the customer,The customer should pay TF + all due amounts i...,عدم التصفح وسوء الخدمه والرجاء اتخاذ قرار ...,System Administrator,NaN,NaN
2,Super 4G on the go 100GB new,CBU,Commercial,Mobile,Commercial claim,CC_Offers,Embedded Bundle - Didn't Get The Monthly Bundles,Family Mobile-IEW,Resolved,Yes,...,Mobile_BO,1,Actual Complaint,SMS,IEW offer -no link,the line is active now & linked & getting the ...,Activation date Sep 22 2022 لم يتم اضافة ا...,NaN,NaN,NaN
3,Pro Bitstream,CBU,Technical,Internet,Technical complaint,TC_Network,SNW-ADSL-Individual fault,Internet SME,Resolved,No,...,NaN,1,User Not Filled,NaN,NaN,NaN,GAIA,NaN,NaN,NaN
4,FTTH Home,CBU,Technical,Internet,Technical complaint,TC_Network,SNW-FTTH - Down interrupted,FTTH Home,Resolved,No,...,NaN,1,User Not Filled,NaN,NaN,NaN,Interrupted,NaN,NaN,NaN


In [4]:
df.shape

(276292, 24)

In [5]:
df.isnull().sum()

OFFER_NAME                 12430
CUSTOMER_TYPE                  4
COMPLAINT_TYPE                 1
PRODUCT                        0
CASE_LEVEL_1                   5
CASE_LEVEL_2                   6
CASE_LEVEL_3                   6
CUSTOMER_GROUP             12228
CURRENT_STATUS                 0
ESCALATION_FLAG                0
CASE_ID                        0
ESCALATED_GROUP           189317
OPEN_DATE                      0
CLOSE_DATE                  2287
CLOSE_GROUP               136362
AGE_BRACKET                 2287
ACTUAL_COMPLAINT               0
CALLBACK_MECHANISM        192986
RESOLUTION                192897
RESOLUTION_DESCRIPTION    192957
CASE_DESC                      0
OPEN_GR                   135114
Unnamed: 22               276268
Unnamed: 23               276265
dtype: int64

# Text Mining

## Text Preprocessing

In [6]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string
import langid

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\USER/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
pip install tashaphyne

Note: you may need to restart the kernel to use updated packages.


In [10]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tashaphyne.stemming import ArabicLightStemmer

def preprocess_arabic_text(text):
    # Remove diacritics
    text = remove_diacritics(text)
    
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenization
    words = word_tokenize(text)
    
    # Remove stop words
    stop_words = set(stopwords.words('arabic'))
    words = [word for word in words if word.lower() not in stop_words]
    
    # Stemming
    stemmer = ArabicLightStemmer()
    words = [stemmer.light_stem(word) for word in words]
    
    # Join the processed words back into a string
    processed_text = ' '.join(words)
    
    return processed_text

def remove_diacritics(text):
    arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(arabic_diacritics, '', str(text))
    return text

In [11]:
import pandas as pd
import re
import langid
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tashaphyne.stemming import ArabicLightStemmer
from nltk.stem import PorterStemmer
import nltk

stop_words_english = set(stopwords.words('english'))

stop_words_arabic = set(stopwords.words('arabic'))

def preprocess_text(text, language):
    if language == 'en':
        # English pre-processing
        tokens = nltk.word_tokenize(text)
        tokens = [token.lower() for token in tokens if re.match(r'\w', token)]
        tokens = [token for token in tokens if token not in stop_words_english]
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(token) for token in tokens]
    elif language == 'ar':
        # Arabic pre-processing
        text = remove_diacritics(text)
        text = re.sub(r'[^\w\s]', '', text)
        words = word_tokenize(text)
        words = [word for word in words if word.lower() not in stop_words_arabic]
        stemmer = ArabicLightStemmer()
        words = [stemmer.light_stem(word) for word in words]
        tokens = words
    else:
        tokens = nltk.word_tokenize(text)

    processed_text = ' '.join(tokens)
    return processed_text

def remove_diacritics(text):
    arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(arabic_diacritics, '', str(text))
    return text

for index, row in df.iterrows():
    text = row['CASE_DESC']
    lang, _ = langid.classify(text)
    processed_text = preprocess_text(text, lang)
    df.at[index, 'preprocessed_text'] = processed_text

In [42]:
df.to_csv('preprocessed_data.csv', index=False)

## Calculate TF-IDF

In [39]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

user_query = input("Enter your query: ")

preprocessed_query = preprocess_text(user_query, langid.classify(user_query)[0])

combined_text = df['CASE_DESC'].append(pd.Series([user_query]))

# Create TF-IDF matrix
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(combined_text)

# Calculate cosine similarity between the user query and the documents
cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])

# Get the indices of the top 5 most similar documents
most_similar_indices = np.argsort(cosine_similarities[0])[:-6:-1]

results_df = pd.DataFrame({
    'Original Document': [df.loc[index, 'CASE_DESC'] for index in most_similar_indices],
    'Cosine Similarity': [cosine_similarities[0][index] for index in most_similar_indices]
})

print("Top 5 most similar original documents:")
print(results_df)

results_df.to_csv('similar_documents_results.csv', index=False)

Enter your query: game
Top 5 most similar original documents:
                                   Original Document  Cosine Similarity
0                                          from game           0.900524
1                             game high ping solved            0.652722
2  NA GOOD SPEED TEST    12.8 - 8  FROM 24  ( GAM...           0.556876
3  kindly remove all customer service as premium ...           0.514515
4           NAF GOOD SOLVEDPUBG GAME --EUROPE SERVER           0.446179


In [37]:
from collections import defaultdict
def count_co_occurrences(preprocessed_texts, co_occurrences_dict):
    for text in preprocessed_texts:
        words = text.split()

        for i in range(len(words)):
            # Check if the word is already in the dictionary
            if words[i] not in co_occurrences_dict:
                co_occurrences_dict[words[i]] = defaultdict(int)

            for j in range(i + 1, len(words)):
                # Check if the word is already in the inner dictionary
                if words[j] not in co_occurrences_dict[words[i]]:
                    co_occurrences_dict[words[i]][words[j]] = 0

                co_occurrences_dict[words[i]][words[j]] += 1

co_occurrences_dict = defaultdict(dict)

# Extract preprocessed text column of the top 5 similar documents
top_5_indices = most_similar_indices  
preprocessed_texts_top_5 = df.loc[top_5_indices, 'preprocessed_text'].tolist()

# Count co-occurrences
count_co_occurrences(preprocessed_texts_top_5, co_occurrences_dict)

print(co_occurrences_dict)

defaultdict(<class 'dict'>, {'مشكل': defaultdict(<class 'int'>, {'انترنت': 5, 'شبك': 1, 'تغط': 2}), 'انترنت': defaultdict(<class 'int'>, {}), 'شبك': defaultdict(<class 'int'>, {'انترنت': 1}), 'تغط': defaultdict(<class 'int'>, {'انترنت': 2}), 'وجد': defaultdict(<class 'int'>, {'مشكل': 1, 'انترنت': 1})})
